In [1]:
# !git clone https://github.com/Arman-Rayan-Sharif/arman-text-emotion.git
# !cp /content/arman-text-emotion/dataset/test.tsv /content/test.tsv
# !cp /content/arman-text-emotion/dataset/train.tsv /content/train.tsv
# !rm -rf /content/arman-text-emotion

Cloning into 'arman-text-emotion'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 52 (delta 10), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 572.39 KiB | 1.91 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
# !pip install openai
# !pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=13108e55caffc81f8ab807ddd45023cf58d0c98b955e652754fc6bafa1257ae0
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [6]:
# !pip uninstall googletrans
# !pip install httpx==0.28.1  # đúng version cho OpenAI 1.x

Found existing installation: googletrans 4.0.0rc1
Uninstalling googletrans-4.0.0rc1:
  Would remove:
    /usr/local/bin/translate
    /usr/local/lib/python3.11/dist-packages/googletrans-4.0.0rc1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/googletrans/*
Proceed (Y/n)? y
  Successfully uninstalled googletrans-4.0.0rc1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


In [1]:
import pandas as pd

train = pd.read_csv("/content/train.tsv", header=None, sep="\t")
test = pd.read_csv("/content/test.tsv", header=None, sep="\t")

train.columns = ['text', 'label']
test.columns = ['text', 'label']

print(train.head())

print("---------------------------------------------------------------------")

print(test.head())

                                                text  label
0  خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...    SAD
1     از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم   HATE
2   "کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم...    SAD
3  چون همش با دوربین ثبت شده ، ایا میشه اعتراض زد...  OTHER
4              اين وضع ب طرز خنده داري گريه داره ...    SAD
---------------------------------------------------------------------
                                                text  label
0  اين شايد اولين عزاى عمومى واقعى است كه ياد دار...    SAD
1  دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...  HAPPY
2   کدوم شعبه پول نداده، بگو الان برات آمار دقیق ...  OTHER
3  امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...  HAPPY
4  امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...    SAD


In [2]:
print(train['label'].value_counts())
print("---------------------------------------------------------------------")
print(test['label'].value_counts())

label
OTHER       1681
ANGRY        923
SAD          896
FEAR         757
SURPRISE     739
HAPPY        618
HATE         511
Name: count, dtype: int64
---------------------------------------------------------------------
label
HAPPY       275
SAD         262
OTHER       193
ANGRY       154
SURPRISE    145
HATE         65
FEAR         57
Name: count, dtype: int64


In [3]:
n_test_sample = 1
test_prompt = train.groupby('label').apply(lambda x: x.sample(n_test_sample))

test_prompt.head()

/tmp/ipython-input-3-90216820.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_prompt = train.groupby('label').apply(lambda x: x.sample(n_test_sample))


,,text,label
label,,,
ANGRY,3190,به وضعیت پیچیدهی ایران که فکر میکنم میریزم به ...,ANGRY
FEAR,5313,واقعا ترسناک,FEAR
HAPPY,5789,ممنون زیبا جان,HAPPY
HATE,368,نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ...,HATE
OTHER,2330,دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ای...,OTHER


In [4]:
n_sample_each_group = 3
demonstration = train.groupby('label').apply(lambda x: x.sample(n_sample_each_group))
demonstration = demonstration.reset_index(drop=True)
demonstration = demonstration.sample(frac=1).reset_index(drop=True)

demonstration.head()

/tmp/ipython-input-4-2600897119.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  demonstration = train.groupby('label').apply(lambda x: x.sample(n_sample_each_group))


,text,label
0,آقا منم یه نور نارنجی عجیب دیدم و گرخیدم …,SURPRISE
1,ترم سه دانشگاه ۸ تا ۱۰ دومیدانی داشتیم۱۰ تا ۱۲...,HATE
2,مرسی عزیزوم,HAPPY
3,الان چه رنگیه ؟,SURPRISE
4,آقایون ترسو جنابان مسئول اونایی که میخواین سر ...,FEAR


### OpenAI Prompt Methodology

In [ ]:
# import openai
# openai.api_key  = "--"

# def get_completion(prompt, model="gpt-3.5-turbo"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0,
#     )
#     return response.choices[0].message["content"]

# def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=temperature
#     )
#     return response.choices[0].message["content"]

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="--")

def get_completion(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

### Prompts

In [6]:
import time

max_retries = 5

### Zero-shot Sample Test

In [7]:
prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: SAD
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: ANGRY
Actual Label: OTHER
--------------------------------------------------

In [8]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: FEAR, ANGER
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: ANGRY
Actual Label: OTHER
------------------------------------------

In [9]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: FEAR, ANXIOUS
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: ANGRY
Actual Label: OTHER
----------------------------------------

In [10]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: FEAR, ANXIOUS
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: ANGRY
Actual Label: OTHER
----------------------------------------

In [11]:
# !pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
# from googletrans import Translator

# prompt = f"""
# The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
# Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
# Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

# Tweet: SAMPLE_TWEET
# Emotion:
# """

# for index, row in test_prompt.iterrows():
#     retries = 0
#     while retries < max_retries:
#         try:
#             translator = Translator()
#             result = translator.translate(row['text'], src='fa', dest='en')

#             response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
#             print(f"Tweet: {row['text']}")
#             print(f"Model Response: {response}")
#             print(f"Actual Label: {row['label']}")
#             print("---------------------------------------------------------------------")
#             break
#         except Exception as e:
#             print(f"An error occurred: {str(e)}")
#             retries += 1
#             print(f"Retrying ({retries}/{max_retries})...")
#             time.sleep(30)

#     if retries == max_retries:
#         print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

In [11]:
from deep_translator import GoogleTranslator

prompt_template = """
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            translated_text = GoogleTranslator(source='fa', target='en').translate(row['text'])
            prompt = prompt_template.replace("SAMPLE_TWEET", translated_text)

            response = get_completion(prompt)
            print(f"Tweet: {row['text']}")
            print(f"Translated: {translated_text}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Translated: My nervousness is going to be a complex situation in Iran that I think I think! I can't even find a temporary or long -term solution that I like to move on to it ...
Model Response: FEAR
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Translated: Really scary
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Translated: Thank you beautiful John
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Translated: Race, religion, ethnic pride, and natio

### Zero-shot Metric

In [12]:
acceptable_classes = [
    "OTHER",
    "ANGRY",
    "SAD",
    "FEAR",
    "SURPRISE",
    "HAPPY",
    "HATE"
]

In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=1)
    precision = precision_score(actual, predicted, average='weighted', zero_division=1)
    recall = recall_score(actual, predicted, average='weighted', zero_division=1)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = set(actual + predicted)

    metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'Class-wise Counts': {}
    }

    for class_label in classes:
        class_precision = precision_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_recall = recall_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)
        class_f1 = f1_score(actual, predicted, labels=[class_label], average='micro', zero_division=1)

        metrics['Class-wise Counts'][class_label] = {
            'Actual Count': actual_counts[class_label],
            'Predicted Count': predicted_counts[class_label],
            'Precision': class_precision,
            'Recall': class_recall,
            'F1 Score': class_f1
        }

    for metric, value in metrics.items():
        if metric == 'Class-wise Counts':
            print(f"\n{metric}:")
            for class_label, stats in value.items():
                print(f"Class: {class_label}")
                for stat, val in stats.items():
                    print(f"{stat}: {val}")
                print()
        else:
            print(f"{metric}: {value}")

    return metrics

In [32]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

100%|██████████| 1151/1151 [07:10<00:00,  2.67it/s]


In [33]:
calculate_metrics(actual, predicted)

Accuracy: 0.5838401390095569
F1 Score: 0.5572762776475274
Precision: 0.6774199802663923
Recall: 0.5838401390095569

Class-wise Counts:
Class: SURPRISE
Actual Count: 145
Predicted Count: 23
Precision: 0.9130434782608695
Recall: 0.14482758620689656
F1 Score: 0.25

Class: SAD
Actual Count: 262
Predicted Count: 199
Precision: 0.7788944723618091
Recall: 0.5916030534351145
F1 Score: 0.6724511930585684

Class: ANGRY
Actual Count: 154
Predicted Count: 338
Precision: 0.3905325443786982
Recall: 0.8571428571428571
F1 Score: 0.5365853658536586

Class: HATE
Actual Count: 65
Predicted Count: 149
Precision: 0.28859060402684567
Recall: 0.6615384615384615
F1 Score: 0.40186915887850466

Class: FEAR
Actual Count: 57
Predicted Count: 55
Precision: 0.7272727272727273
Recall: 0.7017543859649122
F1 Score: 0.7142857142857143

Class: HAPPY
Actual Count: 275
Predicted Count: 326
Precision: 0.7484662576687117
Recall: 0.8872727272727273
F1 Score: 0.8119800332778702

Class: OTHER
Actual Count: 193
Predicted Count:

{'Accuracy': 0.5838401390095569,
 'F1 Score': 0.5572762776475274,
 'Precision': 0.6774199802663923,
 'Recall': 0.5838401390095569,
 'Class-wise Counts': {'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 23,
   'Precision': 0.9130434782608695,
   'Recall': 0.14482758620689656,
   'F1 Score': 0.25},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 199,
   'Precision': 0.7788944723618091,
   'Recall': 0.5916030534351145,
   'F1 Score': 0.6724511930585684},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 338,
   'Precision': 0.3905325443786982,
   'Recall': 0.8571428571428571,
   'F1 Score': 0.5365853658536586},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 149,
   'Precision': 0.28859060402684567,
   'Recall': 0.6615384615384615,
   'F1 Score': 0.40186915887850466},
  'FEAR': {'Actual Count': 57,
   'Predicted Count': 55,
   'Precision': 0.7272727272727273,
   'Recall': 0.7017543859649122,
   'F1 Score': 0.7142857142857143},
  'HAPPY': {'Actual Count': 275,
   'Pr

In [34]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

100%|██████████| 1151/1151 [07:32<00:00,  2.55it/s]


In [35]:
calculate_metrics(actual, predicted)

Accuracy: 0.49782797567332754
F1 Score: 0.49087738363865546
Precision: 0.6110792211254849
Recall: 0.49782797567332754

Class-wise Counts:
Class: SURPRISE
Actual Count: 145
Predicted Count: 26
Precision: 0.8846153846153846
Recall: 0.15862068965517243
F1 Score: 0.26900584795321636

Class: SAD
Actual Count: 262
Predicted Count: 108
Precision: 0.7962962962962963
Recall: 0.3282442748091603
F1 Score: 0.4648648648648649

Class: ANGRY
Actual Count: 154
Predicted Count: 179
Precision: 0.41899441340782123
Recall: 0.487012987012987
F1 Score: 0.45045045045045046

Class: HATE
Actual Count: 65
Predicted Count: 140
Precision: 0.3142857142857143
Recall: 0.676923076923077
F1 Score: 0.4292682926829268

Class: FEAR
Actual Count: 57
Predicted Count: 44
Precision: 0.7045454545454546
Recall: 0.543859649122807
F1 Score: 0.6138613861386139

Class: HAPPY
Actual Count: 275
Predicted Count: 336
Precision: 0.7142857142857143
Recall: 0.8727272727272727
F1 Score: 0.7855973813420621

Class: OTHER
Actual Count: 193
P

{'Accuracy': 0.49782797567332754,
 'F1 Score': 0.49087738363865546,
 'Precision': 0.6110792211254849,
 'Recall': 0.49782797567332754,
 'Class-wise Counts': {'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 26,
   'Precision': 0.8846153846153846,
   'Recall': 0.15862068965517243,
   'F1 Score': 0.26900584795321636},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 108,
   'Precision': 0.7962962962962963,
   'Recall': 0.3282442748091603,
   'F1 Score': 0.4648648648648649},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 179,
   'Precision': 0.41899441340782123,
   'Recall': 0.487012987012987,
   'F1 Score': 0.45045045045045046},
  'HATE': {'Actual Count': 65,
   'Predicted Count': 140,
   'Precision': 0.3142857142857143,
   'Recall': 0.676923076923077,
   'F1 Score': 0.4292682926829268},
  'FEAR': {'Actual Count': 57,
   'Predicted Count': 44,
   'Precision': 0.7045454545454546,
   'Recall': 0.543859649122807,
   'F1 Score': 0.6138613861386139},
  'HAPPY': {'Actual Coun

In [14]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

100%|██████████| 1151/1151 [07:39<00:00,  2.50it/s]


In [15]:
calculate_metrics(actual, predicted)

Accuracy: 0.46568201563857514
F1 Score: 0.4593223118487312
Precision: 0.6181385905687314
Recall: 0.46568201563857514

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 415
Precision: 0.21927710843373494
Recall: 0.47150259067357514
F1 Score: 0.2993421052631579

Class: SURPRISE
Actual Count: 145
Predicted Count: 25
Precision: 0.84
Recall: 0.14482758620689656
F1 Score: 0.24705882352941178

Class: ANGRY
Actual Count: 154
Predicted Count: 155
Precision: 0.3548387096774194
Recall: 0.35714285714285715
F1 Score: 0.3559870550161812

Class: SAD
Actual Count: 262
Predicted Count: 76
Precision: 0.8421052631578947
Recall: 0.24427480916030533
F1 Score: 0.378698224852071

Class: HAPPY
Actual Count: 275
Predicted Count: 311
Precision: 0.752411575562701
Recall: 0.850909090909091
F1 Score: 0.7986348122866894

Class: HATE
Actual Count: 65
Predicted Count: 136
Precision: 0.3382352941176471
Recall: 0.7076923076923077
F1 Score: 0.4577114427860697

Class: FEAR
Actual Count: 57
Predicted Coun

{'Accuracy': 0.46568201563857514,
 'F1 Score': 0.4593223118487312,
 'Precision': 0.6181385905687314,
 'Recall': 0.46568201563857514,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 415,
   'Precision': 0.21927710843373494,
   'Recall': 0.47150259067357514,
   'F1 Score': 0.2993421052631579},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 25,
   'Precision': 0.84,
   'Recall': 0.14482758620689656,
   'F1 Score': 0.24705882352941178},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 155,
   'Precision': 0.3548387096774194,
   'Recall': 0.35714285714285715,
   'F1 Score': 0.3559870550161812},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 76,
   'Precision': 0.8421052631578947,
   'Recall': 0.24427480916030533,
   'F1 Score': 0.378698224852071},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 311,
   'Precision': 0.752411575562701,
   'Recall': 0.850909090909091,
   'F1 Score': 0.7986348122866894},
  'HATE': {'Actual Count': 65,
  

In [ ]:
# from tqdm import tqdm
# from googletrans import Translator

# actual = []
# predicted = []

# prompt = f"""
# The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
# Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
# Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

# Tweet: SAMPLE_TWEET
# Emotion:
# """

# for index, row in tqdm(test.iterrows(), total=len(test)):
#     retries = 0
#     while retries < max_retries:
#         try:
#             translator = Translator()
#             result = translator.translate(row['text'], src='fa', dest='en')

#             response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
#             actual.append(row['label'])

#             if response not in acceptable_classes:
#               predicted.append('OTHER')
#             else:
#               predicted.append(response)

#             break
#         except Exception as e:
#             if "JSON" not in str(e):
#               print(f"An error occurred: {str(e)}")
#               print(f"Retrying ({retries + 1}/{max_retries})...")

#             retries += 1
#             time.sleep(10)

#     if retries == max_retries:
#         print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

In [16]:
from tqdm import tqdm
from deep_translator import GoogleTranslator
import time

actual = []
predicted = []

prompt_template = """
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            # Dùng deep-translator thay cho googletrans
            translated_text = GoogleTranslator(source='fa', target='en').translate(row['text'])

            prompt = prompt_template.replace("SAMPLE_TWEET", translated_text)
            response = get_completion(prompt)
            actual.append(row['label'])

            # Làm sạch kết quả nếu có thêm dấu xuống dòng hoặc khoảng trắng
            response_clean = response.strip().upper()
            if response_clean not in acceptable_classes:
                predicted.append('OTHER')
            else:
                predicted.append(response_clean)
            break

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(10)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

100%|██████████| 1151/1151 [32:13<00:00,  1.68s/it]


In [17]:
calculate_metrics(actual, predicted)

Accuracy: 0.5360556038227628
F1 Score: 0.5290269498699591
Precision: 0.5814921408723659
Recall: 0.5360556038227628

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 239
Precision: 0.3389121338912134
Recall: 0.41968911917098445
F1 Score: 0.375

Class: SURPRISE
Actual Count: 145
Predicted Count: 33
Precision: 0.6363636363636364
Recall: 0.14482758620689656
F1 Score: 0.23595505617977527

Class: ANGRY
Actual Count: 154
Predicted Count: 218
Precision: 0.38990825688073394
Recall: 0.551948051948052
F1 Score: 0.45698924731182794

Class: SAD
Actual Count: 262
Predicted Count: 186
Precision: 0.7258064516129032
Recall: 0.5152671755725191
F1 Score: 0.6026785714285714

Class: HAPPY
Actual Count: 275
Predicted Count: 297
Precision: 0.7205387205387206
Recall: 0.7781818181818182
F1 Score: 0.7482517482517482

Class: HATE
Actual Count: 65
Predicted Count: 128
Precision: 0.359375
Recall: 0.7076923076923077
F1 Score: 0.47668393782383417

Class: FEAR
Actual Count: 57
Predicted Count: 50
Pr

{'Accuracy': 0.5360556038227628,
 'F1 Score': 0.5290269498699591,
 'Precision': 0.5814921408723659,
 'Recall': 0.5360556038227628,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 239,
   'Precision': 0.3389121338912134,
   'Recall': 0.41968911917098445,
   'F1 Score': 0.375},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 33,
   'Precision': 0.6363636363636364,
   'Recall': 0.14482758620689656,
   'F1 Score': 0.23595505617977527},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 218,
   'Precision': 0.38990825688073394,
   'Recall': 0.551948051948052,
   'F1 Score': 0.45698924731182794},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 186,
   'Precision': 0.7258064516129032,
   'Recall': 0.5152671755725191,
   'F1 Score': 0.6026785714285714},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 297,
   'Precision': 0.7205387205387206,
   'Recall': 0.7781818181818182,
   'F1 Score': 0.7482517482517482},
  'HATE': {'Actual Count': 65,
 

### Few-shot Sample Test

In [18]:
def create_demonstration_prompt(demonstration_df, title="Example",
                                input="Tweet", label="Emotion",
                                label_map=None):
  result = ""
  i = 1
  for index, row in demonstration_df.iterrows():
    result += f"{title} {i}:\n"
    result += f"{input}: {row['text']}\n"
    if label_map is None:
      result += f"{label}: {row['label']}\n"
    else:
      result += f"{label}: {label_map[row['label']]}\n"
    i += 1
  return result

In [19]:
print(create_demonstration_prompt(demonstration))

Example 1:
Tweet: آقا منم یه نور نارنجی عجیب دیدم و گرخیدم …
Emotion: SURPRISE
Example 2:
Tweet: ترم سه دانشگاه ۸ تا ۱۰ دومیدانی داشتیم۱۰ تا ۱۲ شنا ۱ تا ۳ کبدی ۴ تا ۶ والیبال. تازه بعدش باید میومدیم خابگاه رخت چرکامونو میشستیم و درسای تئوری فردارو میخوندیم و غذا درست میکردیم و میخوردیم  وقتی راجب کلاسای صبح مجازی ناله میزنید ازتون متنفر میشم 
Emotion: HATE
Example 3:
Tweet: مرسی عزیزوم  
Emotion: HAPPY
Example 4:
Tweet:  الان چه رنگیه ؟
Emotion: SURPRISE
Example 5:
Tweet: آقایون ترسو جنابان مسئول اونایی که میخواین سر راه انقلاب اسلامی سنگبندازین ومیندارین هه... وقتی وزرشگاه ۱۲هزار نفری روبرای دکتر سلام وحضور ماتحویل نمیدین وبرنامه روسعی میکنید لغوکنید منتظر حضور ۱۰۰هزار نفری امت حزب الله در ورزشگاه باشید #شکست_تحریم #اجتماع_دشمن_شکن
Emotion: FEAR
Example 6:
Tweet: کوفتت شه 
Emotion: ANGRY
Example 7:
Tweet: اگر تونستید از ایران فرار کنید ولی هنوز احساس خشم دارید، این مقاله کمکتون میکنه، ولی اگر مثل من داخل ایران هستید همون عصبانی باشید راحت ترید. چون خشم تنها چیزیه که براتون باقی مونده 

In [20]:
prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: SAD
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: HATE
Actual Label: OTHER
---------------------------------------------------

In [21]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: FEAR
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: HATE
Actual Label: OTHER
--------------------------------------------------

In [22]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: ANGRY
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: HATE
Actual Label: OTHER
-------------------------------------------------

In [23]:
prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration, title='مثال', input="توییت", label="احساس")}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: ANGRY
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: HATE
Actual Label: OTHER
-------------------------------------------------

In [24]:
emotion_persian_to_english = {
    "دیگر": "OTHER",
    "عصبانی": "ANGRY",
    "غمگین": "SAD",
    "ترس": "FEAR",
    "غافلگیری": "SURPRISE",
    "خوشحالی": "HAPPY",
    "نفرت": "HATE"
}

emotion_english_to_persian = {
    "OTHER": "دیگر",
    "ANGRY": "عصبانی",
    "SAD": "غمگین",
    "FEAR": "ترس",
    "SURPRISE": "غافلگیری",
    "HAPPY": "خوشحالی",
    "HATE": "نفرت"
}

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration, label_map=emotion_english_to_persian)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            if response not in emotion_persian_to_english.keys():
              print(f"Model Response: NotFound {response}")
            else:
              print(f"Model Response: {emotion_persian_to_english[response]}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: ANGRY
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: SAD
Actual Label: OTHER
--------------------------------------------------

In [25]:
emotion_persian_to_english = {
    "دیگر": "OTHER",
    "عصبانی": "ANGRY",
    "غمگین": "SAD",
    "ترس": "FEAR",
    "غافلگیری": "SURPRISE",
    "خوشحالی": "HAPPY",
    "نفرت": "HATE"
}

emotion_english_to_persian = {
    "OTHER": "دیگر",
    "ANGRY": "عصبانی",
    "SAD": "غمگین",
    "FEAR": "ترس",
    "SURPRISE": "غافلگیری",
    "HAPPY": "خوشحالی",
    "HATE": "نفرت"
}

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration, title='مثال', input="توییت", label="احساس", label_map=emotion_english_to_persian)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            print(f"Tweet: {row['text']}")
            if response not in emotion_persian_to_english.keys():
              print(f"Model Response: NotFound {response}")
            else:
              print(f"Model Response: {emotion_persian_to_english[response]}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Model Response: ANGRY
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Model Response: HATE
Actual Label: HATE
---------------------------------------------------------------------
Tweet: دیوان عالی چرا اوباما پشت پرده پولهای بلوکه ایران را پس داد   #Stolen_works_of_art
Model Response: SAD
Actual Label: OTHER
--------------------------------------------------

In [ ]:
# from googletrans import Translator

# prompt = f"""
# The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
# Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
# Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

# Examples:
# {create_demonstration_prompt(demonstration)}

# Tweet: SAMPLE_TWEET
# Emotion:
# """

# for index, row in test_prompt.iterrows():
#     retries = 0
#     while retries < max_retries:
#         try:
#             translator = Translator()
#             result = translator.translate(row['text'], src='fa', dest='en')

#             response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
#             print(f"Tweet: {row['text']}")
#             print(f"Model Response: {response}")
#             print(f"Actual Label: {row['label']}")
#             print("---------------------------------------------------------------------")
#             break
#         except Exception as e:
#             print(f"An error occurred: {str(e)}")
#             retries += 1
#             print(f"Retrying ({retries}/{max_retries})...")
#             time.sleep(30)

#     if retries == max_retries:
#         print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

In [26]:
from deep_translator import GoogleTranslator
import time

prompt_template = f"""
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in test_prompt.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            # Dịch từ tiếng Persian sang tiếng Anh
            translated_text = GoogleTranslator(source='fa', target='en').translate(row['text'])

            # Thay thế tweet vào prompt
            prompt = prompt_template.replace("SAMPLE_TWEET", translated_text)

            # Gọi API
            response = get_completion(prompt)

            # In kết quả
            print(f"Tweet: {row['text']}")
            print(f"Translated: {translated_text}")
            print(f"Model Response: {response}")
            print(f"Actual Label: {row['label']}")
            print("---------------------------------------------------------------------")
            break

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

Tweet: به وضعیت پیچیدهی ایران که فکر میکنم میریزم به هم، اعصابم خورد میشه! نمیتونم حتی یه راه حل موقتی یا طولانی مدت براش پیدا کنم که دلمو خوش کنم که دارم به سمتش حرکت میکنم...
Translated: My nervousness is going to be a complex situation in Iran that I think I think! I can't even find a temporary or long -term solution that I like to move on to it ...
Model Response: FEAR
Actual Label: ANGRY
---------------------------------------------------------------------
Tweet:  واقعا ترسناک
Translated: Really scary
Model Response: FEAR
Actual Label: FEAR
---------------------------------------------------------------------
Tweet: ممنون زیبا جان 
Translated: Thank you beautiful John
Model Response: HAPPY
Actual Label: HAPPY
---------------------------------------------------------------------
Tweet: نژاد ، دین ، غرور قومی و ملی گرایی هیچ تاثیری ندارند ، به جز اینکه یاد می دهند تا از مردمی که هیچوقت ملاقات نکرده اید متنفر باشید ...! #داگ_استنهوپ
Translated: Race, religion, ethnic pride, and natio

### Few-shot Metric

In [27]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
The below sentence is a person's tweet. The tweet is in Persian. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 92%|█████████▏| 1056/1151 [08:36<00:42,  2.21it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 198874, Requested 1147. Please try again in 6ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


100%|██████████| 1151/1151 [09:45<00:00,  1.97it/s]


In [28]:
calculate_metrics(actual, predicted)

Accuracy: 0.5682015638575152
F1 Score: 0.5742848483089128
Precision: 0.7019293576485341
Recall: 0.5682015638575152

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 109
Precision: 0.6146788990825688
Recall: 0.3471502590673575
F1 Score: 0.44370860927152317

Class: SURPRISE
Actual Count: 145
Predicted Count: 18
Precision: 0.9444444444444444
Recall: 0.11724137931034483
F1 Score: 0.2085889570552147

Class: ANGRY
Actual Count: 154
Predicted Count: 142
Precision: 0.4647887323943662
Recall: 0.42857142857142855
F1 Score: 0.44594594594594594

Class: SAD
Actual Count: 262
Predicted Count: 230
Precision: 0.7608695652173914
Recall: 0.6679389312977099
F1 Score: 0.7113821138211383

Class: HAPPY
Actual Count: 275
Predicted Count: 292
Precision: 0.8116438356164384
Recall: 0.8618181818181818
F1 Score: 0.8359788359788359

Class: HATE
Actual Count: 65
Predicted Count: 315
Precision: 0.1746031746031746
Recall: 0.8461538461538461
F1 Score: 0.2894736842105263

Class: FEAR
Actual Count: 57


{'Accuracy': 0.5682015638575152,
 'F1 Score': 0.5742848483089128,
 'Precision': 0.7019293576485341,
 'Recall': 0.5682015638575152,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 109,
   'Precision': 0.6146788990825688,
   'Recall': 0.3471502590673575,
   'F1 Score': 0.44370860927152317},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 18,
   'Precision': 0.9444444444444444,
   'Recall': 0.11724137931034483,
   'F1 Score': 0.2085889570552147},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 142,
   'Precision': 0.4647887323943662,
   'Recall': 0.42857142857142855,
   'F1 Score': 0.44594594594594594},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 230,
   'Precision': 0.7608695652173914,
   'Recall': 0.6679389312977099,
   'F1 Score': 0.7113821138211383},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 292,
   'Precision': 0.8116438356164384,
   'Recall': 0.8618181818181818,
   'F1 Score': 0.8359788359788359},
  'HATE': {'Actual 

In [29]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 22%|██▏       | 254/1151 [01:46<08:22,  1.78it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 198828, Requested 1221. Please try again in 14ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


 48%|████▊     | 558/1151 [04:42<05:01,  1.97it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 199772, Requested 1170. Please try again in 282ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


 79%|███████▉  | 907/1151 [08:05<02:10,  1.87it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 198866, Requested 1243. Please try again in 32ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


 97%|█████████▋| 1120/1151 [10:09<00:15,  1.98it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 199284, Requested 1161. Please try again in 133ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


100%|██████████| 1151/1151 [10:50<00:00,  1.77it/s]


In [30]:
calculate_metrics(actual, predicted)

Accuracy: 0.5890529973935708
F1 Score: 0.5975176493144487
Precision: 0.6924597457797187
Recall: 0.5890529973935708

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 155
Precision: 0.5483870967741935
Recall: 0.44041450777202074
F1 Score: 0.4885057471264368

Class: SURPRISE
Actual Count: 145
Predicted Count: 32
Precision: 0.9375
Recall: 0.20689655172413793
F1 Score: 0.3389830508474576

Class: ANGRY
Actual Count: 154
Predicted Count: 141
Precision: 0.475177304964539
Recall: 0.43506493506493504
F1 Score: 0.4542372881355932

Class: SAD
Actual Count: 262
Predicted Count: 211
Precision: 0.7914691943127962
Recall: 0.6374045801526718
F1 Score: 0.7061310782241015

Class: HAPPY
Actual Count: 275
Predicted Count: 293
Precision: 0.7986348122866894
Recall: 0.850909090909091
F1 Score: 0.823943661971831

Class: HATE
Actual Count: 65
Predicted Count: 263
Precision: 0.20532319391634982
Recall: 0.8307692307692308
F1 Score: 0.32926829268292684

Class: FEAR
Actual Count: 57
Predicted Coun

{'Accuracy': 0.5890529973935708,
 'F1 Score': 0.5975176493144487,
 'Precision': 0.6924597457797187,
 'Recall': 0.5890529973935708,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 155,
   'Precision': 0.5483870967741935,
   'Recall': 0.44041450777202074,
   'F1 Score': 0.4885057471264368},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 32,
   'Precision': 0.9375,
   'Recall': 0.20689655172413793,
   'F1 Score': 0.3389830508474576},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 141,
   'Precision': 0.475177304964539,
   'Recall': 0.43506493506493504,
   'F1 Score': 0.4542372881355932},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 211,
   'Precision': 0.7914691943127962,
   'Recall': 0.6374045801526718,
   'F1 Score': 0.7061310782241015},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 293,
   'Precision': 0.7986348122866894,
   'Recall': 0.850909090909091,
   'F1 Score': 0.823943661971831},
  'HATE': {'Actual Count': 65,
   '

In [31]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

Examples:
{create_demonstration_prompt(demonstration)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

 34%|███▍      | 394/1151 [02:48<07:19,  1.72it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 198827, Requested 1215. Please try again in 12ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


 48%|████▊     | 555/1151 [04:21<04:31,  2.19it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 198796, Requested 1245. Please try again in 12ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


 74%|███████▍  | 856/1151 [07:16<02:33,  1.92it/s]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on tokens per min (TPM): Limit 200000, Used 198862, Requested 1169. Please try again in 9ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


100%|██████████| 1151/1151 [22:34<00:00,  1.18s/it]


In [32]:
calculate_metrics(actual, predicted)

Accuracy: 0.5516941789748045
F1 Score: 0.5670860194602971
Precision: 0.6758069810408804
Recall: 0.5516941789748045

Class-wise Counts:
Class: OTHER
Actual Count: 193
Predicted Count: 88
Precision: 0.5568181818181818
Recall: 0.2538860103626943
F1 Score: 0.3487544483985765

Class: SURPRISE
Actual Count: 145
Predicted Count: 47
Precision: 0.7872340425531915
Recall: 0.25517241379310346
F1 Score: 0.3854166666666667

Class: ANGRY
Actual Count: 154
Predicted Count: 141
Precision: 0.4326241134751773
Recall: 0.3961038961038961
F1 Score: 0.4135593220338983

Class: SAD
Actual Count: 262
Predicted Count: 179
Precision: 0.8491620111731844
Recall: 0.5801526717557252
F1 Score: 0.6893424036281179

Class: HAPPY
Actual Count: 275
Predicted Count: 303
Precision: 0.7887788778877888
Recall: 0.8690909090909091
F1 Score: 0.8269896193771626

Class: HATE
Actual Count: 65
Predicted Count: 336
Precision: 0.17261904761904762
Recall: 0.8923076923076924
F1 Score: 0.2892768079800499

Class: FEAR
Actual Count: 57
Pre

{'Accuracy': 0.5516941789748045,
 'F1 Score': 0.5670860194602971,
 'Precision': 0.6758069810408804,
 'Recall': 0.5516941789748045,
 'Class-wise Counts': {'OTHER': {'Actual Count': 193,
   'Predicted Count': 88,
   'Precision': 0.5568181818181818,
   'Recall': 0.2538860103626943,
   'F1 Score': 0.3487544483985765},
  'SURPRISE': {'Actual Count': 145,
   'Predicted Count': 47,
   'Precision': 0.7872340425531915,
   'Recall': 0.25517241379310346,
   'F1 Score': 0.3854166666666667},
  'ANGRY': {'Actual Count': 154,
   'Predicted Count': 141,
   'Precision': 0.4326241134751773,
   'Recall': 0.3961038961038961,
   'F1 Score': 0.4135593220338983},
  'SAD': {'Actual Count': 262,
   'Predicted Count': 179,
   'Precision': 0.8491620111731844,
   'Recall': 0.5801526717557252,
   'F1 Score': 0.6893424036281179},
  'HAPPY': {'Actual Count': 275,
   'Predicted Count': 303,
   'Precision': 0.7887788778877888,
   'Recall': 0.8690909090909091,
   'F1 Score': 0.8269896193771626},
  'HATE': {'Actual Coun

In [33]:
from tqdm import tqdm

actual = []
predicted = []

prompt = f"""
جمله زیر توییت یک شخص است. توییت به زبان فارسی است. عواطف یا احساسات مرتبط با آن را شناسایی کنید. \
پاسخ های ممکن عبارتند از: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
زمانی که جمله شامل هیچ احساس خاصی نیست، دارای عواطف مختلط یا دارای احساساتی غیر از کلاس های ذکر شده است، از OTHER استفاده کنید. \

مثال ها:
{create_demonstration_prompt(demonstration)}

توییت: SAMPLE_TWEET
احساس:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            response = get_completion(prompt.replace("SAMPLE_TWEET", row['text']))
            actual.append(row['label'])

            if response not in acceptable_classes:
              predicted.append('OTHER')
            else:
              predicted.append(response)

            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(30)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

  2%|▏         | 26/1151 [02:24<1:06:29,  3.55s/it]

An error occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-lPtVGzVSiGj0YOxZPtL8TTV1 on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying (1/5)...


  3%|▎         | 39/1151 [04:44<2:15:23,  7.31s/it]


KeyboardInterrupt: 

In [ ]:
calculate_metrics(actual, predicted)

In [ ]:
# from tqdm import tqdm
# from googletrans import Translator

# actual = []
# predicted = []

# prompt = f"""
# The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
# Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
# Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

# Examples:
# {create_demonstration_prompt(demonstration)}

# Tweet: SAMPLE_TWEET
# Emotion:
# """

# for index, row in tqdm(test.iterrows(), total=len(test)):
#     retries = 0
#     while retries < max_retries:
#         try:
#             translator = Translator()
#             result = translator.translate(row['text'], src='fa', dest='en')

#             response = get_completion(prompt.replace("SAMPLE_TWEET", result.text))
#             actual.append(row['label'])

#             if response not in acceptable_classes:
#               predicted.append('OTHER')
#             else:
#               predicted.append(response)

#             break
#         except Exception as e:
#             if "JSON" not in str(e):
#               print(f"An error occurred: {str(e)}")
#               print(f"Retrying ({retries+1}/{max_retries})...")
#             retries += 1
#             time.sleep(10)

#     if retries == max_retries:
#         print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

In [ ]:
from tqdm import tqdm
from deep_translator import GoogleTranslator
import time

actual = []
predicted = []

prompt_template = f"""
The below sentence is a person's tweet. The tweet was originally in Persian and I am giving you their translation in English. Identify the emotion or sentiment associated with it. \
Possible answers are: OTHER, ANGRY, SAD, FEAR, SURPRISE, HAPPY, HATE. \
Use OTHER when the sentence does not include any specific emotion, has mixed emotion, or has emotion other than the classes mentioned. \

Examples:
{create_demonstration_prompt(demonstration)}

Tweet: SAMPLE_TWEET
Emotion:
"""

for index, row in tqdm(test.iterrows(), total=len(test)):
    retries = 0
    while retries < max_retries:
        try:
            # Translate from Persian to English
            translated_text = GoogleTranslator(source='fa', target='en').translate(row['text'])

            # Generate prompt
            prompt = prompt_template.replace("SAMPLE_TWEET", translated_text)

            # Get model response
            response = get_completion(prompt)

            actual.append(row['label'])

            # Normalize or validate prediction
            if response.strip() not in acceptable_classes:
                predicted.append('OTHER')
            else:
                predicted.append(response.strip())

            break  # Success → break retry loop

        except Exception as e:
            if "JSON" not in str(e):
                print(f"An error occurred: {str(e)}")
                print(f"Retrying ({retries + 1}/{max_retries})...")
            retries += 1
            time.sleep(10)

    if retries == max_retries:
        print(f"Failed to process tweet after {max_retries} retries. Skipping to the next tweet.")

In [ ]:
calculate_metrics(actual, predicted)